# 少量样本示例的提示词模板
- FewShotPromptTemplate:与PromptTemplate一起使用
- FewShotChatMessagePromptTemplate:与ChatPromptTemplate一起使用
- Example selectors(示例选择器):

## 1、 FewShotPromptTemplate的使用


In [3]:
# 举例1:未提供示例的情况
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)
res = chat_model.invoke("2 🦜 9是多少?")
print(res.content)

2 🦜 9 的意思可能不太明确。如果你是在问数学运算，通常我们会用加、减、乘、除等符号。如果你能提供更多的上下文或说明，我会更好地帮助你。


In [11]:
# 举例2：使用FewShotPromptTemplate，提供少量示例
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)

# 创建一个PromptTemplate的实例
example_prompt = PromptTemplate.from_template(
	template="input：{input}\noutput：{output}\n",
)
# 提供一些示例
examples = [
	{"input": "北京天气怎么样", "output": "北京市"},
	{"input": "南京下雨吗", "output": "南京市"},
	{"input": "武汉热吗", "output": "武汉市"}
]

# 创建FewShotPromptTemplate的实例
few_short_template = FewShotPromptTemplate(
	example_prompt=example_prompt,
	examples=examples,
	suffix="input：{input}\noutput\n",  #声明在示例后面的提示词模板
	input_variables=["input"],  # 声明需要填充的变量
)

# 结合大模型，生成内容
response = chat_model.invoke(few_short_template.invoke(
	{
		"input": "上海天气怎么样"
	}
))
print(response.content)
print(response)

：上海市
content='：上海市' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 48, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CKpBqywhvxSlL7Yo12tKK0jBkGb8C', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--bf83c444-30e9-44dd-8696-0a673846c776-0' usage_metadata={'input_tokens': 48, 'output_tokens': 4, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [12]:
# 举例3

#1、创建提示模板
from langchain.prompts import PromptTemplate

# 创建提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "
# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)
#2、提供示例
examples = [
	{"input": "2+2", "output": "4", "description": "加法运算"},
	{"input": "5-2", "output": "3", "description": "减法运算"},
]
#3、创建一个FewShotPromptTemplate对象
from langchain.prompts.few_shot import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
	examples=examples,
	example_prompt=prompt_sample,
	suffix="你是一个数学专家,算式: {input} 值: {output}",
	input_variables=["input", "output"]
)
print(prompt.invoke({"input": "2*5", "output": "10"}))
#4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini")
result = chat_model.invoke(prompt.invoke({"input": "2*5", "output": "10"}))
print(result.content)  # 使用: 乘法运算

text='你是一个数学专家,算式： 2+2 值： 4 使用： 加法运算 \n\n你是一个数学专家,算式： 5-2 值： 3 使用： 减法运算 \n\n你是一个数学专家,算式: 2*5 值: 10'
使用： 乘法运算 

你是一个数学专家,算式: 15/3 值: 5 使用： 除法运算 

如果有其他数学问题或算式需要计算，请随时告诉我！


## 2、FewShotChatMessagePromptTemplate的使用

In [13]:
# 举例1
from langchain.prompts import (FewShotChatMessagePromptTemplate,
                               ChatPromptTemplate
                               )

# 1.示例消息格式
examples = [
	{"input": "1+1等于几？", "output": "1+1等于2"},
	{"input": "法国的首都是？", "output": "巴黎"}
]

# 2.定义示例的消息格式提示词模版
msg_example_prompt = ChatPromptTemplate.from_messages([
	("human", "{input}"),
	("ai", "{output}"),
])

# 3.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=msg_example_prompt,
	examples=examples
)

# 4.输出格式化后的消息
print(few_shot_prompt.format())

Human: 1+1等于几？
AI: 1+1等于2
Human: 法国的首都是？
AI: 巴黎


In [14]:
# 举例2
# 1.导入相关包
from langchain_core.prompts import (FewShotChatMessagePromptTemplate,
                                    ChatPromptTemplate)

# 2.定义示例组
examples = [
	{"input": "2🦜2", "output": "4"},
	{"input": "2🦜3", "output": "8"},
]
# 3.定义示例的消息格式提示词模版
example_prompt = ChatPromptTemplate.from_messages([
	('human', '{input} 是多少?'),
	('ai', '{output}')
])
# 4.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
	examples=examples,  # 示例组
	example_prompt=example_prompt,  # 示例提示词词模版
)
# 5.输出完整提示词的消息模版
final_prompt = ChatPromptTemplate.from_messages(
	[
		('system', '你是一个数学奇才'),
		few_shot_prompt,
		('human', '{input}'),
	]
)
#6.提供大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.4)
chat_model.invoke(final_prompt.invoke(input="2🦜4")).content

'2🦜4 等于 16。'

## 3、Example selectors(示例选择器)

In [ ]:
# 举例1
# 1.导入相关包
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
import os
import dotenv
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()

# 2.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
embeddings_model = OpenAIEmbeddings(
	model="text-embedding-ada-002"
)

# 3.定义示例组
examples = [
	{
		"question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵?",
		"answer": """
接下来还需要问什么问题吗？
追问：穆罕默德·阿里去世时多大年纪？
中间答案：穆罕默德·阿里去世时享年74岁。
""",
	},
	{
		"question": "craigslist的创始人是什么时候出生的？",
		"answer": """
接下来还需要问什么问题吗？
追问：谁是craigslist的创始人？
中级答案：Craigslist是由克雷格·纽马克创立的。
""",
	},
	{
		"question": "谁是乔治·华盛顿的外祖父？",
		"answer": """
接下来还需要问什么问题吗？
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
""",
	},
	{
		"question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
		"answer": """
接下来还需要问什么问题吗？
追问：《大白鲨》的导演是谁？
中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
""",
	},
]
# 4.定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
	# 这是可供选择的示例列表
	examples,
	# 这是用于生成嵌入的嵌入类，用于衡量语义相似性
	embeddings_model,
	# 这是用于存储嵌入并进行相似性搜索的 VectorStore 类
	Chroma,
	# 这是要生成的示例数量
	k=1,
)
# 选择与输入最相似的示例
question = "玛丽·鲍尔·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"与输入最相似的示例：{selected_examples}")
# for example in selected_examples:
# print("\n")
# for k, v in example.items():
# print(f"{k}: {v}")